# BASICS 03 - Training Classifier with MLR (or Histogram Comparison)

# TODO:
- make a dataset builder from the output of HOTS
- implement a classifier with MLR in torch
- implement a classifier with HC in torch
- implement an accuracy measure

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd '../HOTS'

from network import network
from tools import HOTS_Dataset, get_loader
import tonic.datasets as datasets
import tonic.transforms as transforms
import os

/home/INT/grimaldi.a/Documents/projets/HOTS/newhots/HOTS


In [2]:
train = True
jitter = None
original_dataset = datasets.POKERDVS(save_to='../../Data', train=train, transform = transforms.NumpyAsType(int))

In [3]:
#for poker DVS
sensor_size = original_dataset.sensor_size
name = 'homhots'
tau = (0.7,1.4,2.8)
homeo = (2,1)
timestr = '2021-12-17_poker'

In [4]:
hots = network(name = name, tau = tau, homeo = homeo, timestr = timestr)
path_to_dataset = f'../Records/output/train/{hots.get_fname()}_{len(original_dataset)}_{jitter}/'
print(f'{path_to_dataset} \n does it exists? -> {os.path.exists(path_to_dataset)}')

../Records/output/train/2021-12-17_poker_homhots_(2, 1)_[4, 8, 16]_[0.7, 1.4, 2.8]_[2, 4, 8]_48_None/ 
 does it exists? -> True


In [5]:
dataset = HOTS_Dataset(path_to_dataset, sensor_size, transform=)

In [6]:
loader, order, classes = get_loader(dataset)

In [35]:
import torch
from tqdm import tqdm
def train_MLR(loader, classes, sensor_size, name_model, verbose=True):
    
    num_workers = 0
    learning_rate = 0.005
    beta1, beta2 = 0.9, 0.999
    betas = (beta1, beta2)
    num_epochs = 2 ** 5 + 1
    
    torch.set_default_tensor_type("torch.DoubleTensor")
    criterion = torch.nn.BCELoss(reduction="mean")
    amsgrad = True #or False gives similar results
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if verbose:
        print(f'device -> {device} - num workers -> {num_workers}')

    N = sensor_size[0]*sensor_size[1]*sensor_size[2]
    n_classes = len(classes)
    logistic_model = LRtorch(N, n_classes)
    logistic_model = logistic_model.to(device)
    logistic_model.train()
    optimizer = torch.optim.Adam(
        logistic_model.parameters(), lr=learning_rate, betas=betas, amsgrad=amsgrad
    )
    pbar = tqdm(total=int(num_epochs))
    for epoch in range(int(num_epochs)):
        losses = []
        for X, label in loader:
            X, label = X.to(device), label.to(device)
            X, label = X.squeeze(0), label.squeeze(0) # just one digit = one batch

            print(X.shape)
            outputs = logistic_model(X)

            n_events = X.shape[0]
            labels = label*torch.ones(n_events).type(torch.LongTensor).to(device)
            labels = torch.nn.functional.one_hot(labels, num_classes=n_classes).type(torch.DoubleTensor).to(device)

            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())

        pbar.update(1)

    pbar.close()
    with open(name_model, 'wb') as file:
        pickle.dump([logistic_model, losses], file, pickle.HIGHEST_PROTOCOL)

    return logistic_model, losses

class LRtorch(torch.nn.Module):
    #torch.nn.Module -> Base class for all neural network modules
    def __init__(self, N, n_classes, bias=True):
        super(LRtorch, self).__init__()
        self.linear = torch.nn.Linear(N, n_classes, bias=bias)
        self.nl = torch.nn.Softmax(dim=1)

    def forward(self, factors):
        return self.nl(self.linear(factors))

In [36]:
model_name = f'../Records/models/{hots.get_fname()}_{len(original_dataset)}_{jitter}_LR.pkl'
model, loss = train_MLR(loader, classes, sensor_size, model_name)

device -> cuda - num workers -> 0




  0%|                                                                                                                                                                                                                                                  | 0/33 [00:00<?, ?it/s]

torch.Size([122, 4])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (122x4 and 2450x4)